# Lecture Notes

Instructions:

> **Week k (week that group topics are presented in lecture)** <br>
By Friday at 4pm, the group is required to submit their typed-up notes from that week’s
lecture. This will form the basic outline of their deliverable and they will receive
feedback from the TA on their work. <br><br>
The group should also have an outline of the one outside paper that they intend to
describe in greater detail.

## L13. Constraint Programming

### Big picture

Constraint programming (CP) is a powerful tull to solve combinatorial optimization problems (e.g. planning, scheduling system design etc). Constraints arise in most areas of human endeavour. They basically formalize the relation among several unknowns (or variables), each taking a value in a given domain. The constraint thus restricts the possible values that variables can take, it represents partial information about the variables of interest. For instance, _"The three angles of a triangle sum to 180 degrees"_ relates each angle of the triangle without precisely specifying their values.

Constraint programming is the study of computational systems based on constraints. The idea of constraint programming is to solve problems by stating constraints (requirements) about the problem area and, consequently, finding solution satisfying all the constraints.

A Constraint Satisfaction Problems is usually referred as **CSP**, but it is worth noticing that any CSP can be trasformed in an optimization problem by simpy adding an objective function.

### Viewing Sodoku and N-Queens as CSPs

**N-Queens** The N-queens problem is a well know puzzle among computer scientists. Given any integer $N$, the problem is to place $N$ queens on a $N\times N$ chessboard satisfying the constraint that no two queens threaten each other.

<img src="images/nqueens.png" align="center"/>

Where a queen threatens any other queens on the same row, column and diagonal.

**Sudoku** Another well known puzzle that can be easly solved as CPSs is the Sudoku puzzle. Given a $9\times 9$ grid partially filled with numbers from $1$ to $9$, the goal is to fill the grid entirely such that:

- each $3\times 3$ sub-grid contains all number fomr $1$ to $9$ once
- each row contains all number fomr $1$ to $9$ once
- each column contains all number fomr $1$ to $9$ once

<table><tr>
<td> <img src="images/start_sudoku.png"/> </td>
<td> <img src="images/end_sudoku.png" /> </td>
</tr></table>

### How CSPs are (formally) defined

With the two examples from above it is easy to understand how a constraint programming problem is defined. A **CSP** is defined by the triple $\langle X,\,D,\,C \rangle$, where
- $X=\{x_1,x_2,\ldots,x_n\}$ is the set of _variables_
- $D=\{D_1,D_2,\ldots,D_n\}$ is the set of _variable domains_, where $x_i\in D_i$ for each $i\in\{ 1,\ldots,n\}$
- $C=\{C_1,C_2,\ldots,C_k\}$ is the set of _constraints_
    - A Constraint $C_i = \langle S_i,\, R_i \rangle$ specifies the allowed variable assignment, where
    - The _scope_ $S_i$ is the set of variables it works on
    - The _relation_ $R_i$ is a set of vilid assignments for $S_i$

### Applications of CSPs

TODO

### Tying to other course content
TODO

## L14. Solving Constraint Programs - Propagation and Basic Search
- Constraint propagation: Achieving arc consistency with AC-1 and AC-3
- Examples on integer domains with algebra
- Complexity of AC-1 and AC-3
- Arc consistency: sound but not complete

## L15. Solving Constraint Programs - Backtrack Search and Forward Checking
- Narrative: show the largest reasonable n-Queens problem that can be solved for various solution methods
- Combining inference with search
    - Why standard search sucks
    - Backtrack search
- Constraint Optimization Problem: Branch & Bound
- Forward Checking: Combining backtracking with **limited** constraint propagation
- Dynamic variable ordering and BT-FC
- "Bucket Elimination", a form of adaptive consistency (i.e., variable elimination)
    - Viewing arc consistency as variable elimination

###### Optional Content, Not Discussed in Lecture
- Adaptive consistency is linear for trees
- Iterative repair
- Conflict-directed back jumping

# Symmetries

<img src="images/escher_2.jpg" align="center"/>
<div style="text-align: right"> M. C. Escher </div>


## Introduction

A CSP often exhibit some symmetries, which are mappings that preserve satisfiability of the CSP. They are particularly harmful when we are looking for all possible valid assignment of a CSP since symmetries have many solutions that are just a symmetric solution of another one.

> **Definition (Symmetry).** For any CSP instance $P = \langle X, D, C \rangle$, a solution symmetry of $P$ is a permutation of the set $X\times D$ that preserves the set of solutions to $P$.

In other words, a solution symmetry is a bijective mapping defined on the set of possible variable-value pairs of a CSP, that maps solutions to solutions.

## Why is symmetry important? 

A principal reason for identifying symmetry in CSPs is to reduce search effort by not exploring assignments that are symmetrically equivalent to assignments considered elsewhere in the search. Clearly, if the solution symmetry group is larger than the constraint symmetry group, there will potentially be a greater search reduction from using the solution symmetries, if they can somehow be identified in advance.


## Case Study: symmetries in N-Queens problem

The standard formulation of the N-Queens problem as a CSP has $N$ variables corresponding to the rows of the chessboard, say $Q_1,Q_2,\ldots,Q_n$. The domain of values corresponds to the columns of the chessboard, say $D = \{1, 2,\ldots, n\}$. The constraints can be expressed as follows:
- the values of $Q_1, Q_2, \ldots, Q_n$ are all different; 
- forall $i,j,1\leq i< j \leq N,|Q_i−Q_j \neq |i−j|$.

A chessboard has eight geometric symmetries: 
- horizontal reflection $r_h$
- vertical reflection $r_v$
- reflectins along the two diagonal axes ($r_{d_1}$ and $r_{d_2}$)
- rotations through $90$&deg;, $180$&deg; and $270$&deg ($r_{90}$, $r_{180}$, $r_{270}$)
- identity (no-reflections) $r_I$

We can group all the symmetries in $G = \{r_I, r_h,r_v, r_{d_1}, r_{d_2},r_{90}, r_{180}, r_{270} \}$
For the seek of simplicity let's denote $A^g$, with $g\inG$ as the result of the application of symmetry $g$ to $A$. For example, in the 4-Queens problem $(Q_1 = 2)^{r_{90}} = (Q_2=4)$.

An example can be given by the 4-Queens problem, all the solutions are the following

<img src="images/two-queens_symmetries.png" align="center"/>

It's easy to note that the second solution is the first solution when we flip the chessboard horizontally.

## Avoid symmetries

### Adding Constraints Before Search

Symmetry in CSPs is usually identified, in practice, by applying human insight: the programmer sees that some transformation would transform a hypothetical solution into another hypothetical solution. Then the programmer can try to formalize some constraint that preserve solutions but remove some of the symmetries.

<div class="alert alert-block alert-warning">
<b>TODO:</b> Example of N-Queens problem with broken symmetries through additional constraints
</div>

This approach is fine if done correctly, but can obviously lose solutions if done incorrectly.


### Dynamic Symmetry Breaking Methods

Dynamic symmetry breaking methods are those that operate to break symmetry during the search process, avoiding some brach of the search tree that are symmetrically equivalent to others. Let's explore some of them.

#### Symmetry Breaking During Search (SBDS)

The basic idea of SBDS (this is the name of the algorithm, not a class for them) is to add constraints to a problem so that, after backtracking from a search decision, the SBDS constraints ensure that no symmetric equivalent of that decision is ever allowed. This is a dynamic technique, since we cannot add the constraints until we know what search decision is being made.

<div class="alert alert-block alert-warning">
<b>TODO:</b> Example search tree for SBDS
</div>

A big drawback of this agorithm is that it always demand from the constraint programmer to explicitly state all symmetries $g\inG$a. If a problem has a large number of symmetries there may be too many for the user to identify and implement by hand.

### The Lex-Leader Method

Puget [1] proved that whenever a CSP has symmetry, it is possible to find a _'reduced form'_, with the symmetries eliminated, by adding constraints to the original problem. Puget found such a reduction for three simple constraint problems, and showed that this reduced CSP could be solved more efficiently than in its original form. From this work a method called **lex-leader** was developed, the background idea is rather simple, for each equivalence class of solutions under our symmetry group, we will predefine one to be the canonical solution. We will achieve this by choosing a static variable ordering. 

For example let's consider a problem where we have three variables $x_1$ $x_2$ and $x_3$ subject to `alldifferent` constraint and domain $\{1,2,3\}$. This problem has $3!$ solutions, the easiest way to impose a canonical solution is to impose that $x_1 \leq x_2 \leq x_3$, that is the _lexicographic_ ordering. Adding this additional constraint we will get exactly one solution. All the other can be recovered applying all the symmetries.

In general the lex-leader method impose a variable ordering for Each permutation in the symmetry group, formally

$$ \forall g \in G, \, A\preceq_{\text{lex }} A^g $$

<div class="alert alert-block alert-warning">
<b>TODO:</b> Example of model with/without lex-leader
</div>

An important practical issue with the lex-leader constraints is that they do not “respect” the variable and value ordering heuristics used in search. That is, it may well be that the leftmost solution in the search tree, which would otherwise be found first, is not canonical and so is disallowed, leading to increased search. This is in contrast to techniques such as SBDS, which do respect the heuristic.

#### Double-Lex

A matrix of decision variables has row symmetry iff given a solution, any permutation of the rows is also a solution. Similarly, it has column symmetry iff given a solution, any permutation of the columns is also a solution.  Row and column symmetry occurs in many mod- els with matrices of decision variable. We have seen that that we can add $n!m!$  constraints to break all the symmetries. Adding so many constraints can be counter-productive. A common approach is order the rows of a matrix and **independently** order the columns. This produces only $n + m − 2$ symmetry breaking constraints. This method is called **double-lex** and has the disadvantage of breaking only some of the symmetries.

<div class="alert alert-block alert-warning">
<b>TODO:</b> Example of the same model as before but with double-lex
</div>


## Intractability of Breaking Symmetry


It is worth to mention that lex-leader requires one constraint for each element of the group. 
In the case of a matrix with $m$ rows and $n$ columns, this is $m!n!$, which is impractical in general. Therefore there are many cases where lex-leader is applicable but impractical.

> **Theorem (Walsh 2011)** Given any _simple_ ordering, there exists a symmetry group such that deciding if an as- signment is smallest in its symmetry class according to this ordering is NP-hard.

Since breaking symmetry appears intractable in gen- eral, a major research direction is to identify special cases which occur in practice where the symmetry group is more tractable to break.

> **Theorem (Katsirelos, Narodytska, and Walsh 2010)** Propagating the double-lex constraint is NP-hard.

# References

- [1] J.-F. Puget. On the satisfiability of symmetrical constraint satisfaction problems. In Methodologies for Intelligent Systems (Proceedings of ISMIS’93), LNAI 689, pages 350–361. Springer, 1993.